In [ ]:
#### SGL Greenland - Cloud free landsat images
# Wesley Rancher
# 3 July 2024

# goals for the script:
# 1. read in Rowley's shp file
# 2. Use NDWI to delineate these lakes to get lake perimeter for each of the landsat images (5/28, 6/22, 7/08, 7/31, 8/23) ← perhaps find more 2021 images? 
# 3. export NDWI images // and bands 4, 3, 2 (do we need metadata?)

In [ ]:
### Further analysis after export (prolly in R)
# Use Pope method to estimate lake volume for these 10 lakes for each landsat date
# Sneed/Hamilton do the same (shall we compare two depth-reflectance methods?)
# Using the delineated lake perimeter (from NDWI), extract lake elevations from the DEM (for 10 lakes for each Landsat date)
# Watersheds have been delineated → identify a surface melt model (PDD and ?? energy balance model??) to melt water within the watershed 

In [1]:
# libs and gee setup
import ee 
import geemap

ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
# shpfile and add to map for visual
prospective_lakes = ee.FeatureCollection('users/wesranch/WV01_2021_0721_selected_watersheds_shp')

Map = geemap.Map(center=[70, -50], zoom=5, basemap='Esri.WorldGrayCanvas')

# add to map
Map.addLayer(prospective_lakes, {}, 'wtrshds from NR')
#Map

In [5]:
# defining dates of interest
begin = '2021-05-01'
end = '2021-09-30'

In [1]:
# filter the landsat 8 collection where overlapping with watersheds of interest 
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterDate(begin, end) \
    .filterBounds(prospective_lakes) \
    .filter(ee.Filter.lte('CLOUD_COVER', 10)) \
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5']) \
    .sort('CLOUD_COVER')

# leave this as a shell for images post 2021 
l9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2') \
    .filterDate(begin, end) \
    .filterBounds(prospective_lakes) \
    .filter(ee.Filter.lte('CLOUD_COVER', 10)) \
    .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5']) \
    .sort('CLOUD_COVER')

all_images_sr = l8.merge(l9)

# inspect what the collection returned
print(l8.size().getInfo())
print(l9.size().getInfo())
amt_of_images_sr = all_images_sr.size().getInfo()
#print('number of images:', amt_of_images_sr_l8)
#print('number of images:', amt_of_images_sr_l9)
print('number of images:', amt_of_images_sr)

# what are the dates
image_dates_sr = all_images_sr.aggregate_array('DATE_ACQUIRED').getInfo()
for date in image_dates_sr:
    print(date)

NameError: name 'ee' is not defined

In [25]:
# compute NDWI and add it as a band
def compute_ndwi(image):
    ndwi = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')
    return image.addBands(ndwi)
# compute ndwi ice
def compute_ndwi_ice(image):
    ndwi_ice = image.normalizedDifference(['SR_B2', 'SR_B4']).rename('NDWI_Ice')
    return image.addBands(ndwi_ice)
# compute mndwi
def compute_mndwi(image):
    mndwi = image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')
    return image.addBands(mndwi)
# apply the functions
all_images_w_ndwi = all_images_sr.map(compute_ndwi)
all_images_w_mndwi = all_images_sr.map(compute_mndwi)
all_images_ndwi_ice = all_images_sr.map(compute_ndwi_ice)

# for one image just for quick inspection
one_image = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_008012_20210731')
mndwi = one_image.normalizedDifference(['SR_B3', 'SR_B6']).rename('MNDWI')

In [27]:
# export the one mndwi image  
export_task = ee.batch.Export.image.toDrive(
    image=mndwi.select('MNDWI'),
    description='LC08_008012_20210731_MNDWI',
    folder='gee_SGL_exports',
    scale=30,            
    region=mndwi.geometry().bounds(),
    fileFormat='GeoTIFF')
export_task.start()

In [77]:
# function for export (main goal here is retention of file nomenclature)
# this will be composites instead of images as individual bands only
def export_all_images(image_collection, label):
    image_list = image_collection.toList(image_collection.size())
    image_count = image_collection.size().getInfo()

    for i in range(image_count):
        image = ee.Image(image_list.get(i))
        image_id = image.get('system:index').getInfo()  

        export_task = ee.batch.Export.image.toDrive(
            image=image.select('SR_B2', 'SR_B3', 'SR_B4'),
            description=image_id,
            folder='gee_SGL_exports',
            scale=30,
            region=image.geometry().bounds(),
            fileFormat='GeoTIFF'
        )
        export_task.start()
        print(f"exporting {label}, image {i+1} of {image_count} with ID: {image_id}")

# another function just for the NDWI of each image as it is float datatype
def export_ndwi(image_collection, label):
    image_list = image_collection.toList(image_collection.size())
    image_count = image_collection.size().getInfo()

    for i in range(image_count):
        image = ee.Image(image_list.get(i))
        image_id = image.get('system:index').getInfo()  

        export_task = ee.batch.Export.image.toDrive(
            image=image.select('NDWI'),
            description=f'{image_id}_NDWI',
            folder='gee_SGL_exports',
            scale=30,
            region=image.geometry().bounds(),
            fileFormat='GeoTIFF'
        )
        export_task.start()
        print(f"exporting {label}, image {i+1} of {image_count} with ID: {image_id}")

In [78]:
# apply the function
export_all_images(all_images_w_ndwi, 'raw bands')
export_ndwi(all_images_w_ndwi, 'ndwi')

exporting raw bands, image 1 of 19 with ID: 1_LC08_006013_20210919
exporting raw bands, image 2 of 19 with ID: 1_LC08_007012_20210910
exporting raw bands, image 3 of 19 with ID: 1_LC08_007012_20210926
exporting raw bands, image 4 of 19 with ID: 1_LC08_007012_20210505
exporting raw bands, image 5 of 19 with ID: 1_LC08_007012_20210708
exporting raw bands, image 6 of 19 with ID: 1_LC08_007012_20210622
exporting raw bands, image 7 of 19 with ID: 1_LC08_008012_20210917
exporting raw bands, image 8 of 19 with ID: 1_LC08_006013_20210701
exporting raw bands, image 9 of 19 with ID: 1_LC08_008012_20210901
exporting raw bands, image 10 of 19 with ID: 1_LC08_006013_20210514
exporting raw bands, image 11 of 19 with ID: 1_LC08_009012_20210503
exporting raw bands, image 12 of 19 with ID: 1_LC08_008012_20210731
exporting raw bands, image 13 of 19 with ID: 1_LC08_009011_20210924
exporting raw bands, image 14 of 19 with ID: 1_LC08_009011_20210823
exporting raw bands, image 15 of 19 with ID: 1_LC08_00801